In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from PIL import Image
import numpy as np
import os
from xml.etree import ElementTree as et
from sklearn.model_selection import train_test_split

In [3]:
import zipfile

# Define the path to your zip file
zip_file_path = '/content/archive.zip'

# Define the directory where you want to extract the files
extracted_dir_path = '/content/extracted_files/'

# Create a ZipFile object
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all the contents of the zip file into the specified directory
    zip_ref.extractall(extracted_dir_path)

# Print a message indicating the extraction is complete
print("Extraction complete.")


Extraction complete.


In [5]:
# Define your root directories
train_root = '/content/extracted_files/train_zip/train'
val_root = '/content/extracted_files/test_zip/test'

# Define your labels
labels = ['background', 'orange', 'apple', 'banana']
label2targets = {l: t for t, l in enumerate(labels)}
num_classes = len(labels)

In [7]:
import os
import numpy as np
import xml.etree.ElementTree as et
from PIL import Image

def preprocess_img(img_path):
    img = Image.open(img_path).convert('RGB')
    img = img.resize((224, 224))  # Resize the image to a fixed size
    img = np.array(img)
    return img

def load_dataset(root):
    images = []
    targets = []
    xml_paths = sorted([os.path.join(root, file) for file in os.listdir(root) if file.endswith('.xml')])
    for xml_path in xml_paths:
        img_path = xml_path.replace('.xml', '.jpg')
        img = preprocess_img(img_path)
        tree = et.parse(xml_path)
        root = tree.getroot()
        for obj in root.findall('object'):
            label = obj.find('name').text
            label_id = label2targets.get(label, 0)  # Assign 0 (background) if label not found
            box = obj.find('bndbox')
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            targets.append([xmin, ymin, xmax, ymax, label_id])  # Store targets as a list
            images.append(img)
    return np.array(images), np.array(targets)

# Define your train and validation root directories
train_root = '/content/extracted_files/train_zip/train'
val_root = '/content/extracted_files/train_zip/train'

# Load the dataset
X_train, y_train = load_dataset(train_root)
X_val, y_val = load_dataset(val_root)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images

  warnings.warn(


In [8]:
# Define your model
def get_model(num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = layers.GlobalAveragePooling2D()(base_model.output)
    x = layers.Dense(256, activation='relu')(x)
    outputs = layers.Dense(num_classes)(x)
    model = keras.Model(inputs=base_model.input, outputs=outputs)
    return model

# Compile your model
model = get_model(num_classes)
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

# Train your model
model.fit(X_train, y_train[:, -1], epochs=30, batch_size=32, validation_data=(X_val, y_val[:, -1]))


94765736/94765736 [==============================] - 5s 0us/step

Epoch 1/30

15/15 [==============================] - 13s 401ms/step - loss: 0.6656 - accuracy: 0.7634 - val_loss: 0.2288 - val_accuracy: 0.9054

Epoch 2/30

15/15 [==============================] - 3s 175ms/step - loss: 0.2493 - accuracy: 0.9075 - val_loss: 0.2179 - val_accuracy: 0.9118

Epoch 3/30

15/15 [==============================] - 3s 176ms/step - loss: 0.2804 - accuracy: 0.8968 - val_loss: 0.1907 - val_accuracy: 0.9140

Epoch 4/30

15/15 [==============================] - 3s 179ms/step - loss: 0.2518 - accuracy: 0.9054 - val_loss: 0.2066 - val_accuracy: 0.9097

Epoch 5/30

15/15 [==============================] - 4s 273ms/step - loss: 0.2010 - accuracy: 0.9226 - val_loss: 0.1686 - val_accuracy: 0.9204

Epoch 6/30

15/15 [==============================] - 3s 177ms/step - loss: 0.2191 - accuracy: 0.9161 - val_loss: 0.1944 - val_accuracy: 0.9204

Epoch 7/30

15/15 [==============================] - 3s 178ms/step -

In [9]:
# Evaluate your model
model.evaluate(X_val, y_val[:, -1])

converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model to a file
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)

print('end')

15/15 [==============================] - 1s 95ms/step - loss: 0.1675 - accuracy: 0.9204

end
